In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNetSmall, UNetLarge
from evaluate.cross_evaluator import SemanticCrossEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')

cv_param = {'interval_img_out': 1,
            'num_images': 3,
            'device': th.device("cuda" if th.cuda.is_available() else "cpu"),
            'datasets_path': [ds1_path, ds2_path],
            'results_path': os.path.join(cwd, '../results'),
            'folds': [0, 1, 2, 3],
            'epochs_cv': 1,
            'epochs_ct': 1}

param = {'id': 0,
         'padding_mode': 'reflect',
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': MultiHyperparameter({'type': ['sgd',
                                                    #'adam',
                                                    #'rmsprop',
                                                    #'asgd'
                                                    ],
                                           'lr_factor': [0.5,
                                                         0.75,
                                                         1,
                                                         1.25,
                                                         1.5],
                                           'weight_decay': [0,
                                                            1e-3,
                                                            1e-5]
                                           }).get_full_grid_params(),
         'augment_transform': [{'rotate': False, 'mirror': False, 'translate': False, 'pad': 0},
                               # {'rotate': True, 'mirror': True, 'translate': False, 'pad': 0},
                               # {'rotate': True, 'mirror': True, 'translate': True, 'pad': 16},
                               # {'rotate': True, 'mirror': True, 'translate': True, 'pad': 64}
                               ],
         'num_augments': 100,
         'binarizer_lr': 0.05,
         'batch_size': 1}

unet_hyps = MultiHyperparameter(param)
params = unet_hyps.get_random_params(1)
unet = UNetLarge.__new__(UNetLarge)

In [3]:
cte = SemanticCrossEvaluator(unet, cv_param)
report = cte.cross_test_model(params, cv_param['epochs_ct'], cv_param['epochs_cv'])
#cte.train_validate(params[0], ds1_path, [0, 1, 2], [3], 50)

Testing Fold: 0 / 3
Evaluating Param: 0 / 0
Evaluating Dataset: 0 / 1
Cross validate: 0 / 2


Training: 100%|██████████████████████████████| 1/1 [00:30<00:00, 30.87s/it]

Loss: 0.5063078135251999 Acc: 0.9163636267185211
Cross validate: 1 / 2



Training: 100%|██████████████████████████████| 1/1 [00:28<00:00, 28.91s/it]

Loss: 0.5263882502913475 Acc: 0.959442981264808
Cross validate: 2 / 2



Training: 100%|██████████████████████████████| 1/1 [00:29<00:00, 29.37s/it]

Loss: 0.49571716248989106 Acc: 0.957743105434236
Evaluating Dataset: 1 / 1
Cross validate: 0 / 2



Training: 100%|██████████████████████████████| 1/1 [00:28<00:00, 28.95s/it]

Loss: 0.5301169192790985 Acc: 0.9494940394704993
Cross validate: 1 / 2


KeyboardInterrupt: 